In [1]:
#Input files
work_dir = '/mnt/storage1/adam/CGRP/New/P006/'
input_pdb = 'P006_CGRP-R_bound_split_lipids_charmmwaters.pdb'          #Split Lipid PDB file with no hydrogens
split_lipids_pdb = 'P006_CGRP-R_bound_split_lipids_charmmwaters.pdb'     


#Output file (Don't touch these)
output_pdb1 = 'pdb_renamed.pdb'
output_pdb2 = 'pdb_renamed_lipids_replaced.pdb'
output = 'PDBfile_HIS_need_replacing.pdb'
output_protein_only = 'PDBfile_protein_only.pdb'

In [2]:
with open(work_dir+input_pdb, 'r') as original_file:
    lines = original_file.readlines()
 
with open(work_dir+output_pdb1, 'w') as new_file:
    for line in lines:
        if 'ACE' in line or 'NHE' in line or 'HSD' in line:
            line = line.replace('HETATM', 'ATOM  ')
        if 'HSD' in line:
            line = line.replace('HSD', 'HIS')
        if 'ILE' in line:
            line = line.replace('CD ','CD1')
        if 'OT1' in line:
            line = line.replace('OT1', 'O  ')
        if 'OT2' in line:
            line = line.replace('OT2', 'OXT')
        new_file.write(line)
        if 'NME' in line:
            new_file.write('TER\n')
        if 'OXT' in line:
            new_file.write('TER\n')

In [3]:
source_lines = []
filtered_lines = []
Filtered_Lipids_Water_Salts = []
with open(work_dir + output_pdb1, 'r') as original_file:
    for line in original_file:
        if any(ion in line for ion in ["PA", "PC", "OL", "WAT", "Na+", "Cl-", "TER"]):
            source_lines.append(line)
        else:
            filtered_lines.append(line)

replacement_lines = []
with open(work_dir + split_lipids_pdb, 'r') as replacement_file:
    for line in replacement_file:
        if any(ion in line for ion in ["PA", "PC", "OL", "WAT", "Na+", "Cl-", "TER"]):
            replacement_lines.append(line)
        else:
            Filtered_Lipids_Water_Salts.append(line)

with open(work_dir + output_pdb2, 'w') as output_file:
    for line in filtered_lines:
        output_file.write(line)
    for line in replacement_lines:
        output_file.write(line)

In [4]:
with open(work_dir+output_pdb2, 'r') as infile, open(work_dir+output, 'w') as outfile:
    for line in infile:
        if 'CONECT' not in line and 'END' not in line:
            outfile.write(line)

    outfile.write('TER\n''END\n')

The final part is to open the PDBfile_ready_for_tleap, add a "TER" line before the first PA line and after any lines that contain OXT, and change the C-terminus protecting group to NHE (if needed)

## Get accurate histidine protonation states with pdb2pqr

https://pdb2pqr.readthedocs.io/en/latest/getting.html

https://github.com/Electrostatics/pdb2pqr

In [5]:
exclude_keywords = ["PA", "PC", "OL", "WAT", "Na+", "Cl-","TER","END"]

with open(work_dir+output, 'r') as file:
    filtered_lines = [line for line in file if not any(keyword in line for keyword in exclude_keywords)]

with open(work_dir+output_protein_only, 'w') as file:
    file.writelines(filtered_lines)


In [6]:
import subprocess
output_protein_only = output_protein_only.replace('.pdb', '')

command = [
    "pdb2pqr30",
    f"{work_dir}{output_protein_only}.pdb",
    f"{work_dir}{output_protein_only}.pqr",
    "--pdb-out", f"{work_dir}{output_protein_only}_propka.pdb",
    "--ffout", "AMBER",
    "--titration-state-method", "propka",
    "--with-ph", "7.4"
]

subprocess.run(command)

INFO:PDB2PQR v3.6.2: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: /mnt/storage1/adam/CGRP/New/P006/PDBfile_protein_only.pdb
INFO:Setting up molecule.
INFO:Created biomolecule object with 467 residues and 3804 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 1 missing atoms in biomolecule.
INFO:Added atom OXT to residue PHE F 37 at coordinates -7.957, -5.918, -44.727
INFO:Updating disulfide bridges.
INFO:Debumping biomolecule.
INFO:Assigning titration states with PROPKA.


CompletedProcess(args=['pdb2pqr30', '/mnt/storage1/adam/CGRP/New/P006/PDBfile_protein_only.pdb', '/mnt/storage1/adam/CGRP/New/P006/PDBfile_protein_only.pqr', '--pdb-out', '/mnt/storage1/adam/CGRP/New/P006/PDBfile_protein_only_propka.pdb', '--ffout', 'AMBER', '--titration-state-method', 'propka', '--with-ph', '7.4'], returncode=0)

In [7]:
# Input files
original_file = f"{work_dir}{output_protein_only}.pdb"
propka_file = f"{work_dir}{output_protein_only}_propka.pdb"
output_file = f"{work_dir}PDBfile_ready_for_tleap.pdb"

with open(original_file, 'r') as file:
    original_lines = file.readlines()

residue_map = {}
with open(propka_file, 'r') as file:
    for line in file:
        if line.startswith("ATOM") or line.startswith("HETATM"):
            residue_number = line[22:26].strip()
            residue_name = line[17:20].strip()
            residue_map[residue_number] = residue_name

final_lines = []

for line in original_lines:
    if line.startswith("ATOM") or line.startswith("HETATM"):
        residue_name = line[17:20].strip() 
        residue_number = line[22:26].strip() 

        if residue_name == "HIS" and residue_number in residue_map:
            new_residue_name = residue_map[residue_number] 
            new_line = line[:17] + new_residue_name + line[20:] 
            final_lines.append(new_line) 
        else:
            final_lines.append(line) 
    else:
        final_lines.append(line) 

    if line.startswith("ATOM") or line.startswith("HETATM"):
        if "OXT" in line:
            final_lines.append("TER\n") 

with open(work_dir + split_lipids_pdb, 'r') as replacement_file:
    for line in replacement_file:
        if any(ion in line for ion in ["PA", "PC", "OL", "WAT", "Na+", "Cl-", "TER"]):
            final_lines.append(line) 

final_lines.append("END\n") 

with open(output_file, 'w') as file:
    file.writelines(final_lines)

print(f"Updated PDB file created: {output_file}")


Updated PDB file created: /mnt/storage1/adam/CGRP/New/P006/PDBfile_ready_for_tleap.pdb


### Check the PDBfile_ready_for_tleap.pdb file and double check everything is fine. Things to check include OXT, NHE, NME, TER records, END